## ここで実施すること

既存LLMをファインチューニングすることで、柔軟かつ組織特色を帯びた対話能を実現する。

### 目標

- 質問に対して端的に回答する。

- 語尾に「ってな～。」とつける

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
import torch
import datasets
import random

#### モデルのロード

In [2]:
model_name = "rinna/japanese-gpt2-medium"

## トークナイザー
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    legacy = False
)

## LLM
model = AutoModelForCausalLM.from_pretrained(model_name)

## 演算にコンピュータのGPUを利用する
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=32000, bias=False)
)

In [3]:
def generate_sentence(
    input_messages_series: str,
    tkn,
    mdl
):
    ##
    input_tokens = tkn.encode(
        input_messages_series,
        return_tensors = 'pt',
        add_special_tokens = False
    ).to(mdl.device)
    ##
    output_tokens = mdl.generate(
        input_tokens,
        max_length = 100,
        min_length = 100,
        do_sample = True,
        top_k = 500,
        top_p = 0.95,
        pad_token_id = tkn.pad_token_id,
        bos_token_id = tkn.bos_token_id,
        eos_token_id = tkn.eos_token_id,
        bad_words_ids = [[tkn.unk_token_id]]
    )
    ##
    output_messages_series = tkn.decode(
        output_tokens[0],
        skip_special_tokens = True
    )
    
    ##
    return output_messages_series

In [4]:
%%time

print(generate_sentence(
    "random モジュールを使用するにはどうしたらいいですか？",
    tokenizer,
    model
))

random モジュールを使用するにはどうしたらいいですか? - trigun ガイド このプログラムは、random 関連モジュールを使用するための要件と手順を定義します。このパッケージでは tariff の他に tensorflow 関数や salt ランディングページを指定するモジュールもサポートしています。また、このランディングページは clink 用 tensorflow を実行中、または実行中に問題なく読み込まれた場合にのみ
CPU times: user 1.77 s, sys: 306 ms, total: 2.07 s
Wall time: 2.07 s


#### データセットのロード

In [5]:
ds = datasets.load_dataset("cl-nagoya/auto-wiki-qa")
ds = ds["train"].shuffle().select(range(1000))
ds

Dataset({
    features: ['passage_id', 'query', 'answer', 'text', 'title', 'url'],
    num_rows: 1000
})

In [6]:
ds[0]

{'passage_id': 3524337,
 'query': 'アフリカ系トルコ人はどこに多い？',
 'answer': 'エーゲ海地方',
 'text': '少なからぬアフリカ系トルコ人人口が存在する地域がエーゲ海地方、特に、イズミル県、アイドゥン県、ムーラ県にある。アンタルヤ県とアダナ県の村や町にもアフリカ系黒人を先祖に持つ人々がいる。これらのアフリカ系移住民の中には移住先に残ったり、通婚により他のエスニックグループに同化していったりするグループもあるが、多くの者が、より大きな都会へと二次移住した。また、このことはアフリカ系トルコ人の正確な人口の推定を難しくしている要因でもある。',
 'title': 'アフリカ系トルコ人',
 'url': 'https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%95%E3%83%AA%E3%82%AB%E7%B3%BB%E3%83%88%E3%83%AB%E3%82%B3%E4%BA%BA'}

In [7]:
def dialectize(example):
    ##
    r = random.randint(1, 3)
    if r == 1:
        tail = "ってな！"
    elif r == 2:
        tail = "ってな？？"
    else:
        tail = "ってな～"
        
    ##
    if example["answer"][-1] == "。":
        answer = example["answer"].replace("です。", tail).replace("ます。", tail)
    else:
        answer = example["answer"] + tail
        
    example["answer"] = answer
    return example

ds = ds.map(
    dialectize
)
ds[0]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'passage_id': 3524337,
 'query': 'アフリカ系トルコ人はどこに多い？',
 'answer': 'エーゲ海地方ってな～',
 'text': '少なからぬアフリカ系トルコ人人口が存在する地域がエーゲ海地方、特に、イズミル県、アイドゥン県、ムーラ県にある。アンタルヤ県とアダナ県の村や町にもアフリカ系黒人を先祖に持つ人々がいる。これらのアフリカ系移住民の中には移住先に残ったり、通婚により他のエスニックグループに同化していったりするグループもあるが、多くの者が、より大きな都会へと二次移住した。また、このことはアフリカ系トルコ人の正確な人口の推定を難しくしている要因でもある。',
 'title': 'アフリカ系トルコ人',
 'url': 'https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%95%E3%83%AA%E3%82%AB%E7%B3%BB%E3%83%88%E3%83%AB%E3%82%B3%E4%BA%BA'}

In [8]:
MAX_LENGTH = 256

def serialize(example):
    example["tmp"] = "hhh:\n{0}\nqqq:\n{1}\naaa:\n{2}".format(example["text"], example["query"], example["answer"])
    return tokenizer(
        example["tmp"],
        max_length = MAX_LENGTH,
        padding = 'max_length',
        truncation = True
    )

ds = ds.map(
    serialize,
    remove_columns = ["query", "answer", "text", "title", "url"]
)
ds

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['passage_id', 'tmp', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [9]:
tokenizer.decode(ds[0]['input_ids'])

'hhh: 少なからぬアフリカ系トルコ人人口が存在する地域がエーゲ海地方、特に、イズミル県、アイドゥン県、ムーラ県にある。アンタルヤ県とアダナ県の村や町にもアフリカ系黒人を先祖に持つ人々がいる。これらのアフリカ系移住民の中には移住先に残ったり、通婚により他のエスニックグループに同化していったりするグループもあるが、多くの者が、より大きな都会へと二次移住した。また、このことはアフリカ系トルコ人の正確な人口の推定を難しくしている要因でもある。 qqq: アフリカ系トルコ人はどこに多い? aaa: エーゲ海地方ってな～</s>[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [10]:
ds = ds.shuffle().train_test_split(test_size = 0.1)
ds

DatasetDict({
    train: Dataset({
        features: ['passage_id', 'tmp', 'input_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['passage_id', 'tmp', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [11]:
## チューニングパラメータ

num_train_epochs = 8
per_device_train_batch_size = 8
per_device_eval_batch_size = 4
learning_rate = 1e-4
weight_decay = 0.01

trained_model_name = 'seanayuuto/rinna-1b'

In [12]:
## チューニングの準備

training_args = TrainingArguments(
    output_dir = f"../{trained_model_name}",
    eval_strategy = 'epoch',
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    per_device_eval_batch_size = per_device_eval_batch_size,
    learning_rate = learning_rate,
    weight_decay = weight_decay,
    logging_steps = 100
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = False
)

trainer = Trainer(
    model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = ds['train'],
    eval_dataset = ds['test']
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [13]:
%%time
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.573400,2.313496
2,1.896400,2.343163
3,1.411100,2.459843
4,1.034400,2.576315
5,0.771000,2.659364
6,0.577800,2.710455
7,0.450900,2.740462
8,0.322900,2.763320


CPU times: user 5min 21s, sys: 1min 6s, total: 6min 27s
Wall time: 6min 37s


TrainOutput(global_step=904, training_loss=1.0421991805850932, metrics={'train_runtime': 397.155, 'train_samples_per_second': 18.129, 'train_steps_per_second': 2.276, 'total_flos': 3343322500300800.0, 'train_loss': 1.0421991805850932, 'epoch': 8.0})

In [ ]:
hint = "18、19日実施の毎日新聞世論調査で、選択的夫婦別姓制度を導入することに賛成かどうかを聞いた。「賛成」は42％で、「反対」は23％。「どちらとも言えない」は34％だった。"
question = "選択的夫婦別姓のさん"

print(generate_sentence(
    f"hhh:\n{hint}\nqqq:\n{question}\naaa:\n",
    tokenizer,
    trainer.model
))

In [14]:
# ## huggingface保存

# trainer.model.push_to_hub(
#     private = False,
#     repo_id = trained_model_name
# )